### Import library

In [1]:
import os
os.chdir('../..')
os.getcwd()

'C:\\Users\\csia7\\OneDrive\\문서\\GitHub\\WQBrain_2024_API'

In [2]:
import ace_lib as ace
import helpful_functions as hf
import pandas as pd
import requests
import plotly.express as px
import pygwalker as pyg
import glob
import re

### Start session
Enter credentials once - they will be saved to local folder and loaded each time

In [3]:
s = ace.start_session()

Complete biometrics authentication and press any key to continue: 
https://api.worldquantbrain.com/authentication/persona?inquiry=inq_nNPgdHRpafG1xrrcT8WUdypKpSjD

 


In [4]:
ace.check_session_timeout(s)

14362.158774

## Alpha Mixing Templates

In [5]:
'''
signal1 = scale_down(<cs_operator>(ts_backfill(<data1>,126)));

signal2 = scale_down(<cs_operator>(ts_backfill(<data2>,126)));

signal1 * (1 + signal2)
'''

'\nsignal1 = scale_down(<cs_operator>(ts_backfill(<data1>,126)));\n\nsignal2 = scale_down(<cs_operator>(ts_backfill(<data2>,126)));\n\nsignal1 * (1 + signal2)\n'

## Read all passed alphas

In [6]:
os.chdir('Results/signal')
file_direc = glob.glob('*')
file_direc

['A_signalform.csv',
 'B_signalform.csv',
 'C_signalform.csv',
 'D_signalform.csv',
 'pre_signalform.csv',
 'Sep03GLB_signalform.csv',
 'Sep13GLB_signalform.csv']

In [7]:
expressions = {}
for file in file_direc:
    expressions[file.split('_')[0]] = pd.read_csv(file)['expression']
#expressions

In [8]:
expressions.keys()

dict_keys(['A', 'B', 'C', 'D', 'pre', 'Sep03GLB', 'Sep13GLB'])

In [9]:
expressions['Sep13GLB']

0     s1_Sep13GLB_0 = ts_zscore (ts_backfill (vec_av...
1     s1_Sep13GLB_1 = ts_zscore (ts_backfill (vec_av...
2     s1_Sep13GLB_2 = ts_zscore (ts_backfill (vec_av...
3     s1_Sep13GLB_3 = ts_zscore (ts_backfill (vec_av...
4     s1_Sep13GLB_4 = ts_zscore (ts_backfill (vec_av...
5     s1_Sep13GLB_5 = ts_zscore (ts_backfill (vec_av...
6     s1_Sep13GLB_6 = ts_zscore (ts_backfill (vec_av...
7     s1_Sep13GLB_7 = ts_zscore (ts_backfill (vec_av...
8     s1_Sep13GLB_8 = ts_zscore (ts_backfill (vec_av...
9     s1_Sep13GLB_9 = ts_zscore (ts_backfill (vec_av...
10    s1_Sep13GLB_10 = ts_zscore (ts_backfill (vec_a...
11    s1_Sep13GLB_11 = ts_zscore (ts_backfill (vec_a...
12    s1_Sep13GLB_12 = ts_zscore (ts_backfill (vec_a...
13    s1_Sep13GLB_13 = ts_zscore (ts_backfill (vec_a...
14    s1_Sep13GLB_14 = ts_zscore (ts_backfill (vec_a...
15    s1_Sep13GLB_15 = ts_zscore (ts_backfill (vec_a...
16    s1_Sep13GLB_16 = ts_zscore (ts_backfill (vec_a...
17    s1_Sep13GLB_17 = ts_zscore (ts_backfill (v

## Template Sep03GLB & Sep13GLB mixing
use top 32 results

In [59]:
#region= "GLB", universe = "MINVOL1M", neutralization = 'MARKET', truncation = 0.01

In [12]:
Sep03GLB = expressions['Sep03GLB']
Sep13GLB = expressions['Sep13GLB']

In [23]:
alpha_list_Sep03GLBSep13GLB = []

for i in range(5):
    for j in range(10, 20):
        alpha_list_Sep03GLBSep13GLB.append(ace.generate_alpha(f'{Sep03GLB[i]};{Sep13GLB[j]};add(scale_down(signal_Sep03GLB_{i}), scale_down(signal_Sep13GLB_{j}), filter = false)', region= "GLB", universe = "MINVOL1M", neutralization = 'COUNTRY', truncation = 0.01, decay = 3))

alpha_list_Sep03GLBSep13GLB

[{'type': 'REGULAR',
  'settings': {'instrumentType': 'EQUITY',
   'region': 'GLB',
   'universe': 'MINVOL1M',
   'delay': 1,
   'decay': 3,
   'neutralization': 'COUNTRY',
   'truncation': 0.01,
   'pasteurization': 'ON',
   'testPeriod': 'P0Y0M0D',
   'unitHandling': 'VERIFY',
   'nanHandling': 'OFF',
   'language': 'FASTEXPR',
   'visualization': False},
  'regular': 'group_Sep03GLB_0 = (country+1)*group_max(pv13_52_minvol_1m_all_delay_1_sector, market) + pv13_52_minvol_1m_all_delay_1_sector;signal_Sep03GLB_0 = group_neutralize(ts_scale(group_backfill(fnd23_intfvalld1_ecns, industry, 252, std=1), 252), densify(group_Sep03GLB_0));s1_Sep13GLB_10 = ts_zscore (ts_backfill (vec_avg(mdl139_score), 252), 252);s2_Sep13GLB_10 = ts_zscore (ts_backfill (vec_avg(oth193_wolfe_shield_global_mktcap), 252), 252);group_Sep13GLB_10 = pv13_1l_scibr;signal_Sep13GLB_10 = group_zscore(s1_Sep13GLB_10+s2_Sep13GLB_10, group_Sep13GLB_10);add(scale_down(signal_Sep03GLB_0), scale_down(signal_Sep13GLB_10), filt

In [24]:
result = ace.simulate_alpha_list_multi(s, alpha_list_Sep03GLBSep13GLB)

100%|█████████████████████████████████████████████████████████████████████████████████| 17/17 [49:21<00:00, 174.18s/it]


In [25]:
#prettify_result function can be used from the helpful_functions library to take a look at IS stats of all the simulated alphas

result_st1 = hf.prettify_result(result, detailed_tests_view=False)
result_st1

,pnl,book_size,long_count,short_count,turnover,returns,drawdown,margin,fitness,sharpe,start_date,alpha_id,expression,concentrated_weight,high_turnover,is_ladder_sharpe,low_fitness,low_sharpe,low_sub_universe_sharpe,low_turnover
0,4703552,20000000,3444,3313,0.0464,0.0454,0.0246,0.001959,1.43,2.37,2012-01-22,2L0WA75,group_Sep03GLB_0 = (country+1)*group_max(pv13_...,PASS,PASS,PASS,PASS,PASS,PASS,PASS
1,4709526,20000000,3443,3313,0.0482,0.0455,0.0253,0.001888,1.42,2.36,2012-01-22,wlmPQlv,group_Sep03GLB_0 = (country+1)*group_max(pv13_...,PASS,PASS,PASS,PASS,PASS,PASS,PASS
2,4685885,20000000,3138,2993,0.0507,0.0452,0.0247,0.001785,1.42,2.36,2012-01-22,EE7L6EJ,group_Sep03GLB_0 = (country+1)*group_max(pv13_...,PASS,PASS,PASS,PASS,PASS,PASS,PASS
3,4679427,20000000,3194,3050,0.0494,0.0452,0.0252,0.001831,1.40,2.33,2012-01-22,3Rb5o7Z,group_Sep03GLB_0 = (country+1)*group_max(pv13_...,PASS,PASS,PASS,PASS,PASS,PASS,PASS
4,4776417,20000000,3293,3155,0.0427,0.0461,0.0278,0.002161,1.37,2.25,2012-01-22,k07ONKP,group_Sep03GLB_0 = (country+1)*group_max(pv13_...,PASS,PASS,PASS,PASS,PASS,PASS,PASS
5,4343104,20000000,3443,3316,0.0499,0.0419,0.0312,0.001681,1.25,2.16,2012-01-22,gMAdeeK,group_Sep03GLB_0 = (country+1)*group_max(pv13_...,PASS,PASS,FAIL,PASS,PASS,PASS,PASS
6,4319355,20000000,3442,3315,0.0517,0.0417,0.0303,0.001614,1.25,2.16,2012-01-22,n2JQXVw,group_Sep03GLB_0 = (country+1)*group_max(pv13_...,PASS,PASS,FAIL,PASS,PASS,PASS,PASS
7,4535765,20000000,3292,3157,0.0469,0.0438,0.0334,0.001869,1.23,2.07,2012-01-22,vlq61v3,group_Sep03GLB_0 = (country+1)*group_max(pv13_...,PASS,PASS,FAIL,PASS,PASS,PASS,PASS
8,4272254,20000000,3442,3315,0.0496,0.0413,0.0334,0.001665,1.21,2.11,2012-01-22,d0XoWag,group_Sep03GLB_0 = (country+1)*group_max(pv13_...,PASS,PASS,FAIL,PASS,PASS,PASS,PASS
9,4458939,20000000,3293,3158,0.0465,0.0431,0.0354,0.001851,1.19,2.02,2012-01-22,698JdAY,group_Sep03GLB_0 = (country+1)*group_max(pv13_...,PASS,PASS,FAIL,PASS,PASS,PASS,PASS


In [26]:
result_st1.to_csv('Sep03GLB&Sep13GLB_mixing_add_2.csv')

In [27]:
#to take a look at the combined result of all new alphas

is_tests_df = hf.concat_is_tests(result)
is_tests_df.head()

,alpha_id,endDate,limit,message,name,result,startDate,themes,value,year
0,k07ONKP,NaN,1.58,NaN,LOW_SHARPE,PASS,NaN,NaN,2.2500,NaN
1,k07ONKP,NaN,1.00,NaN,LOW_FITNESS,PASS,NaN,NaN,1.3700,NaN
2,k07ONKP,NaN,0.01,NaN,LOW_TURNOVER,PASS,NaN,NaN,0.0427,NaN
3,k07ONKP,NaN,0.70,NaN,HIGH_TURNOVER,PASS,NaN,NaN,0.0427,NaN
4,k07ONKP,NaN,NaN,NaN,CONCENTRATED_WEIGHT,PASS,NaN,NaN,NaN,NaN


In [28]:
#making a list of failed alphas
failed_alphas = is_tests_df.query('result=="FAIL"')['alpha_id'].unique()

#making a list of passed alphas
passed_alphas = list(set(is_tests_df['alpha_id']).difference(failed_alphas))

print(f'Failed alphas:{failed_alphas}\nPassed alphas:{passed_alphas}')

Failed alphas:['vlq61v3' '698JdAY' 'gMAdeeK' 'd0XoWag' 'n2JQXVw']
Passed alphas:['k07ONKP', '3Rb5o7Z', '2L0WA75', 'EE7L6EJ', 'wlmPQlv']


In [29]:
#calling submit_alpha on all alphas that have passed the submission tests
for alpha_id in passed_alphas:
    hf.set_alpha_properties(s, alpha_id, tags = ['Sep14_GLB_mixing'])

submit tmr!

In [22]:
submit_result = {alpha_id: ace.submit_alpha(s, alpha_id) for alpha_id in passed_alphas[4:]}
submit_result

{'j0EP169': False,
 'xk62RvN': False,
 'd0X3d9Y': False,
 'rl3NAlj': False,
 'JOwWbWl': False,
 '698ozMJ': False}

### Library Fuctions.

following are some other functions that you can use for your own analysis

**get_alpha_pnl(s, alpha_id)** - to get the pnl for an alpha

**get_alpha_yearly_stats(s, alpha_id)** - to get yearly statistics for an alpha

**get_self_corr(s, alpha_id)** - to get self correlation results for an alpha

**get_prod_corr(s, alpha_id)** - to get prod correlation results for an alpha

**get_check_submission(s, alpha_id)** - to get check submission result for an alpha

**check_self_corr_test(s, alpha_id)** - to check if alpha passes self correlation test (self_corr<0.7)

**check_prod_corr_test(s, alpha_id)** - to check if alpha passes prod correlation test (prod_corr<0.7)

**perfomance_comparison(s, alpha_id)** - to get the result of performance comparison for an alpha merged performance

In [37]:
ace.get_alpha_pnl(s, result[0]['alpha_id'])

,Pnl,alpha_id
Date,,
2012-01-23,0.0,dx9ALqJ
2012-01-24,52190.0,dx9ALqJ
2012-01-25,32038.0,dx9ALqJ
2012-01-26,37403.0,dx9ALqJ
2012-01-27,53631.0,dx9ALqJ
...,...,...
2022-01-17,12935472.0,dx9ALqJ
2022-01-18,12935661.0,dx9ALqJ
2022-01-19,12967474.0,dx9ALqJ
